In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image, ImageDraw, ImageFont

image = Image.open("/content/drive/MyDrive/Publaynet/train-0-1/PMC3651331_00005.jpg")
image = image.convert("RGB")
image

In [ ]:
import json
with open('/content/drive/MyDrive/Publaynet/train.json') as f:
  train_json = json.load(f)

In [ ]:
print(train_json.keys())

In [ ]:
print(train_json['annotations'][0].keys())

In [ ]:
import os
img_list = os.listdir('/content/drive/MyDrive/Publaynet/train-0-1')
print(img_list)

In [ ]:
def reletive_position(box1,box2):
  x1,y1 = box1[:2]
  x2,y2 = box2[:2]
  if abs(x1-x2) > abs(y1-y2):
    if x1 > x2:
      return 'right'
    else:
      return 'left'
  else:
    if y1 > y2:
      return 'down'
    else:
      return 'above'

In [ ]:
scene_graph = {}
relation_id = 0
for i in range(1,2):
  img_name = 'PMC3651331_00005.jpg'
  scene_graph[img_name] = {}
  for j in range(len(train_json['images'])):
    image_info = train_json['images'][j]
    if img_name == image_info['file_name']:
      scene_graph[img_name]['image_id'] = image_info['id']
      scene_graph[img_name]['width'] = image_info['width']
      scene_graph[img_name]['height'] = image_info['height']
  scene_graph[img_name]['objects'] = {}
  for j in range(len(train_json['annotations'])):
    box_info = train_json['annotations'][j]
    if scene_graph[img_name]['image_id'] == box_info['image_id']:
      scene_graph[img_name]['objects'][box_info['id']] = {}
      scene_graph[img_name]['objects'][box_info['id']]['name'] = box_info['category_id']
      scene_graph[img_name]['objects'][box_info['id']]['box'] = box_info['bbox']
      scene_graph[img_name]['objects'][box_info['id']]['relations'] = {}
  for box1 in scene_graph[img_name]['objects']:
    for box2 in scene_graph[img_name]['objects']:
      if box1 != box2:
        scene_graph[img_name]['objects'][box1]['relations'][str(relation_id)] = {} 
        bbox1 = scene_graph[img_name]['objects'][box1]['box']
        bbox2 = scene_graph[img_name]['objects'][box2]['box']
        scene_graph[img_name]['objects'][box1]['relations'][str(relation_id)]['name'] = reletive_position(bbox1,bbox2)
        scene_graph[img_name]['objects'][box1]['relations'][str(relation_id)]['object'] = box2
        relation_id += 1

In [ ]:
import json
import os 
path = '/content/'
os.chdir(path)
file_name = 'scene_graph_test_position_relation.json' 
with open(file_name,'w') as file_object:
    json.dump(scene_graph,file_object)
